In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
def extract_features(file_path):
    """ Loads an audio file and returns the average MFCC """
    y, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc_mean


In [ ]:
DATASET_PATH = "/content/drive/MyDrive/faysond/genres_original"
data = []
genres = os.listdir(DATASET_PATH)

for genre in genres:
    genre_path = os.path.join(DATASET_PATH, genre)
    if not os.path.isdir(genre_path):
        continue

    for file in os.listdir(genre_path):
        file_path = os.path.join(genre_path, file)
        features = extract_features(file_path)
        data.append([features, genre])

In [ ]:
df = pd.DataFrame(data, columns=["features", "genre"])
df["features"] = df["features"].apply(lambda x: np.array(x))

encoder = LabelEncoder()
df["genre"] = encoder.fit_transform(df["genre"])

In [ ]:
X = np.stack(df["features"].values)
y = df["genre"].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Dataset split completed!")
print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")


Dataset split completed!
Training set: (799, 13)
Validation set: (100, 13)
Test set: (100, 13)


In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f"\n Random Forest Model Accuracy: {rf_accuracy * 100:.2f}%")
print("\n Classification Report - Random Forest:\n", classification_report(y_test, y_pred_rf))


🎯 Random Forest Model Accuracy: 62.00%

📊 Classification Report - Random Forest:
               precision    recall  f1-score   support

           0       0.62      0.80      0.70        10
           1       0.73      0.80      0.76        10
           2       0.55      0.60      0.57        10
           3       0.57      0.40      0.47        10
           4       0.70      0.70      0.70        10
           5       0.60      0.60      0.60        10
           6       0.67      0.60      0.63        10
           7       0.60      0.90      0.72        10
           8       0.75      0.30      0.43        10
           9       0.50      0.50      0.50        10

    accuracy                           0.62       100
   macro avg       0.63      0.62      0.61       100
weighted avg       0.63      0.62      0.61       100



In [ ]:
mlp_model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(len(np.unique(y)), activation="softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
mlp_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
mlp_model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

loss, mlp_accuracy = mlp_model.evaluate(X_test, y_test)
print(f"\nMLP Model Accuracy: {mlp_accuracy * 100:.2f}%")

Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.1596 - loss: 2.2735 - val_accuracy: 0.3100 - val_loss: 1.9980
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2834 - loss: 1.9670 - val_accuracy: 0.3800 - val_loss: 1.8023
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2942 - loss: 1.8638 - val_accuracy: 0.3800 - val_loss: 1.6843
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3563 - loss: 1.7967 - val_accuracy: 0.3700 - val_loss: 1.6019
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3966 - loss: 1.6385 - val_accuracy: 0.4100 - val_loss: 1.5357
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4269 - loss: 1.6055 - val_accuracy: 0.4600 - val_loss: 1.4774
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4212 - loss: 1.5986 - val_accuracy: 0.4600 - val_loss: 1.4342
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4650 - loss: 1.5045 - val_accuracy: 0.4700 - val_loss